## Описание и реализацию алгоритмов поиска смотрел вот здесь https://habrahabr.ru/post/331192/

In [1]:
import math
import heapq
import numpy as np
import xml.etree.ElementTree as ET

from pprint import pprint

In [2]:
def read_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    params = dict()
    
    def get_grid():
        rows = []
        for i in root.findall("map/grid/row"):
            rows.append([_ for _ in map(int, i.text.split(' '))])
        return rows
    
    # get parametrs from xml
    params["width"] = int(root.find("map/width").text)
    params["height"] = int(root.find("map/height").text)
    params["startx"] = int(root.find("map/startx").text)
    params["starty"] = int(root.find("map/starty").text)
    params["finishx"] = int(root.find("map/finishx").text)
    params["finishy"] = int(root.find("map/finishy").text)
    params["grid"] = get_grid()
    params["searchtype"] = root.find('algorithm/searchtype').text # "astar", "theta", "jp_search", "bfs", "dijkstra"
    params["metrictype"] = root.find('algorithm/metrictype').text # "euclidean", "diagonal", "manhattan", "chebyshev"
    params["breakingties"] = root.find('algorithm/breakingties').text # "g-min", "g-max"
    params["hweight"] = float(root.find('algorithm/hweight').text)
    params["allowdiagonal"] = True if root.find('algorithm/allowdiagonal').text=='true' else False
    params["cutcorners"] = True if root.find('algorithm/cutcorners').text=='true' else False
    params["allowsqueeze"] = True if root.find('algorithm/allowsqueeze').text=='true' else False
    params["loglevel"] = float(root.find('options/loglevel').text)
    params["logpath"] = root.find('options/logpath').text
    params["logfilename"] = root.find('options/logfilename').text
    
    return params

In [3]:
xml_path = "./data/example.xml"
params = read_xml(xml_path)
pprint(params["searchtype"])
pprint(params["grid"])

'astar'
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 

In [4]:
class Grid():
    def __init__(self, width, height, grid):
        self.w = width
        self.h = height
        self.grid = grid
        self.walls = [(i,j) for i in range(len(self.grid))
                            for j in range(len(self.grid[i])) if self.grid[i][j]]
    
    
    def is_inside(self, point):
        return 0 <= point[0] < self.w and 0 <= point[1] < self.h
    
    
    def get_value(self, point):
        return 0 if point not in self.walls else 1
    
    def is_not_wall(self, point):
        return False if self.get_value(point) else True

    
    def is_empty_diagonal(self, from_point, to_point):
        neighbor1 = self.get_value((to_point[0], from_point[1]))
        neighbor2 = self.get_value((from_point[0], to_point[1]))
        return not neighbor1 and not neighbor2
    
    
    def neighbors(self, point, allow_diagonal=False, cut_corners=False, allow_sq=False):
        x = point[0]
        y = point[1]
        
        neighbors = [(x + 1, y),
                     (x - 1, y),
                     (x, y + 1),
                     (x, y - 1)]
        
        diagonal_neighbors = set([(x - 1, y - 1),
                              (x - 1, y + 1),
                              (x + 1, y - 1),
                              (x + 1, y + 1)])
        if not cut_corners:
            for dif_x in [1, -1]:
                for dif_y in [1, -1]:
                    if self.get_value((x + dif_x, y)) and self.get_value((x, y + dif_y)):
                        diagonal_neighbors.discard((x + dif_x, y + dif_y))

        if not allow_sq:
            for dif_x in [1, -1]:
                for dif_y in [1, -1]:
                    if self.get_value((x + dif_x, y)) or self.get_value((x, y + dif_y)):
                        diagonal_neighbors.discard((x + dif_x, y + dif_y))
            
        if allow_diagonal:
            neighbors += list(diagonal_neighbors)
            
        neighbors = list(filter(self.is_inside, neighbors)) 
        neighbors = list(filter(self.is_not_wall, neighbors))
        
        return neighbors
    
    
    def cost(self, start_point, end_point, method='euclidean'):
        return pow(pow((start_point[0] - end_point[0]), 2) + pow((start_point[1] - end_point[1]), 2), 0.5)

In [5]:
class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]

In [6]:
def heuristic(first_point, second_point, method='euclidean'):
    """
    euclidean,
    diagonal,
    manhattan,
    chebyshev
    """
    
    dx = abs(first_point[0] - second_point[0])
    dy = abs(first_point[1] - second_point[1])
    
    if method == 'euclidean':
        result = math.sqrt(dx ** 2 + dy ** 2)
    elif method == 'manhattan':
        result = dx + dy
    elif method == 'diagonal':
        result = (dx + dy) + (math.sqrt(2) - 2) * min(dx, dy)
    elif method == 'chebyshev':
        result = (dx + dy) - min(dx, dy)
        
    return result

In [7]:
def a_star(grid, start_point, end_point,
           allow_diagonal=False, cut_corners=False, allow_sq=False,
           heuristic_method='euclidean'):
    
    C = 0
    frontier = PriorityQueue()
    frontier.put(start_point, 0)
    came_from = {}
    cost_so_far = {}
    came_from[start_point] = None
    cost_so_far[start_point] = 0
    path = [end_point]
    
    while not frontier.empty():
        C += 1
        current_point = frontier.get()

        if current_point == end_point:
            while came_from[current_point] != start_point:
                path.append(came_from[current_point])
                current_point = came_from[current_point]
            path.append(start_point)
            print('Path was found!, C = {}'.format(C))
            return C, path

        for neighbor_point in grid.neighbors(current_point, allow_diagonal, cut_corners, allow_sq):
            new_cost = cost_so_far[current_point] + grid.cost(current_point, neighbor_point)
            if neighbor_point not in cost_so_far or new_cost < cost_so_far[neighbor_point]:
                cost_so_far[neighbor_point] = new_cost
                priority = new_cost + heuristic(end_point, neighbor_point, method=heuristic_method)
                frontier.put(neighbor_point, priority)
                came_from[neighbor_point] = current_point

In [8]:
start_point = (params["starty"], params["startx"])
end_point = (params["finishy"], params["finishx"])
grid = Grid(params["width"], params["height"], params["grid"])

In [9]:
_ = a_star(grid, start_point, end_point)

Path was found!, C = 408


In [10]:
_ = a_star(grid, start_point, end_point, heuristic_method='diagonal')

Path was found!, C = 404


In [11]:
_ = a_star(grid, start_point, end_point, allow_diagonal=True)

Path was found!, C = 527


In [12]:
_ = a_star(grid, start_point, end_point, allow_diagonal=True, cut_corners=True)

Path was found!, C = 527


In [13]:
_ = a_star(grid, start_point, end_point, allow_diagonal=True, cut_corners=True, allow_sq=True)

Path was found!, C = 70


In [14]:
def dijkstra(grid, start_point, end_point,
           allow_diagonal=False, cut_corners=False, allow_sq=False):
    
    C = 0
    frontier = PriorityQueue()
    frontier.put(start_point, 0)
    came_from = {}
    cost_so_far = {}
    came_from[start_point] = None
    cost_so_far[start_point] = 0
    path = [end_point]
    
    while not frontier.empty():
        C += 1
        current_point = frontier.get()

        if current_point == end_point:
            while came_from[current_point] != start_point:
                path.append(came_from[current_point])
                current_point = came_from[current_point]
            path.append(start_point)
            print('Path was found!, C = {}'.format(C))
            return C, path

        for neighbor_point in grid.neighbors(current_point, allow_diagonal, cut_corners, allow_sq):
            new_cost = cost_so_far[current_point] + grid.cost(current_point, neighbor_point)
            if neighbor_point not in cost_so_far or new_cost < cost_so_far[neighbor_point]:
                cost_so_far[neighbor_point] = new_cost
                priority = new_cost
                frontier.put(neighbor_point, priority)
                came_from[neighbor_point] = current_point

In [15]:
_ = dijkstra(grid, start_point, end_point)

Path was found!, C = 519


In [16]:
_ = dijkstra(grid, start_point, end_point, allow_diagonal=True)

Path was found!, C = 563


---

In [17]:
def write_log(name, C, path, grid):
    with open('{}_log.txt'.format(name), 'w') as f:
        f.write("C = {}".format(C))
        f.write("\n")
        
        f.write("\nPath: \n")
        for point in path:
            f.write("{}, {}\n".format(point[0], point[1]))
                    
        f.write("\nGrid:\n")        
        grid_with_path = grid
        for point in path:
            grid_with_path[point[0]][point[1]] = "*"
        for raw in grid:
            f.write(" ".join(map(str, raw)))
            f.write("\n")

In [18]:
C, path = a_star(grid, start_point, end_point, allow_diagonal=True)
write_log('astar', C, path, params["grid"])

Path was found!, C = 527


In [19]:
C, path = dijkstra(grid, start_point, end_point)
write_log('dijkstra', C, path, params["grid"])

Path was found!, C = 519
